# Face Recognition on Yale Faces Dataset

## Importing Libraries

In [2]:
import cv2
import numpy as np
from PIL import Image
import os

## Loading the dataset

In [3]:
dataset_path = "data"

os.listdir(dataset_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data'

## Splitting the dataset to test and train

In [5]:
import shutil

def split_files_test_train(ratio, path):
    files = os.listdir(path)
    num_of_files = len(files)
    num_of_test_files = int(num_of_files * ratio)
    test_files = np.random.choice(files, num_of_test_files, replace=False)
    train_files = [file for file in files if file not in test_files]
    return test_files, train_files

def carry_files_to_folder(files, path, new_path):
    if not os.path.exists(new_path):
        os.makedirs(new_path)

        # print absolute path
        print(f"Directory {os.path.abspath(new_path)} created")
    for file in files:
        file_path = os.path.join(path, file)
        new_file_path = os.path.join(new_path, file)
        shutil.move(file_path, new_file_path)

dataset_path = "data"
test_files, train_files = split_files_test_train(0.1, dataset_path)
carry_files_to_folder(train_files, dataset_path, 'train')
carry_files_to_folder(test_files, dataset_path, 'test')

## Testing directories

In [6]:
# add .gif to each file to make it accessible by Image.open
for file in os.listdir('train'):
    os.rename(f'train/{file}', f'train/{file}.gif')
for file in os.listdir('test'):
    os.rename(f'test/{file}', f'test/{file}.gif')

## Getting image datas

In [7]:
def get_image_data(path):
    paths = [img_file for img_file in os.listdir(path) if img_file.endswith('.gif')]
    faces = []
    ids = []

    for image_path in paths:
        image = Image.open(f'{path}/{image_path}').convert('L')

        image_np = np.array(image, 'uint8')
        image_id = int(image_path.split('.')[0].replace('subject', ''))

        faces.append(image_np)
        ids.append(image_id)

    return np.array(ids), faces

ids, faces = get_image_data('train')

## Training and saving the model

In [9]:
import cv2.face as cv

lbph_classifier = cv.LBPHFaceRecognizer.create()
lbph_classifier.train(faces, np.array(ids))

classifier_yml = 'lbph_classifier.yml'

lbph_classifier.write(classifier_yml)

## Testing the dataset

In [1]:
lbph_face_classifier = cv.LBPHFaceRecognizer.create()
lbph_face_classifier.read(classifier_yml)

# test on test data
def test_lbph_face_classifier(path = 'test'):
    ids, faces = get_image_data(path)

    predictions = []

    for i, face in enumerate(faces):
        label, confidence = lbph_face_classifier.predict(face)
        predictions.append((label, confidence))

predictions = test_lbph_face_classifier()

predictions


NameError: name 'cv' is not defined